In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
os.chdir('/mnt/Storage/home/wangyiman/CHMsInOtherContexts/CellStateTransition/DataCollection')

In [4]:
data_geo_dict = {}
for tissue in ['heart', 'liver'] :
    data_geo_dict[tissue] = pd.read_csv(f'DataCollection_add_{tissue}.csv', header = 0, sep = ",", usecols = ['Stage', 'Data', 'GSMID', 'GSEID'])

data_geo_dict[tissue].head()

,Stage,Data,GSMID,GSEID
0,E11.5 Liver,H3K9me3 ChIP-seq,GSM2192347,GSE82974
1,E11.5 Liver,H3K9me3 ChIP-seq,GSM2192348,GSE82974
2,E12.5 Liver,H3K9me3 ChIP-seq,GSM2191248,GSE82470
3,E12.5 Liver,H3K9me3 ChIP-seq,GSM2191249,GSE82470
4,E13.5 Liver,H3K9me3 ChIP-seq,GSM2191298,GSE82494


In [18]:
sra_meta = pd.read_csv("SraRunTable_ENCODE_mouse_embryo.csv", sep = ",", header = 0, 
                       usecols = ['Run', 'Assay Type', "SRA Study", "GEO_Accession (exp)", 'Tissue', 'dev_stage', 'LibraryLayout'])
sra_meta

,Run,Assay Type,LibraryLayout,SRA Study,GEO_Accession (exp),Tissue,dev_stage
0,SRR10049450,ChIP-Seq,SINGLE,SRP013703,GSM4048835,NaN,NaN
1,SRR10049470,ChIP-Seq,SINGLE,SRP013703,GSM4050822,NaN,embryonic
2,SRR10049471,ChIP-Seq,SINGLE,SRP013703,GSM4050823,NaN,embryonic
3,SRR10049474,ChIP-Seq,SINGLE,SRP013703,GSM4050830,NaN,NaN
4,SRR10049475,ChIP-Seq,SINGLE,SRP013703,GSM4050831,NaN,NaN
...,...,...,...,...,...,...,...
4347,SRR16812120,DNase-Hypersensitivity,PAIRED,SRP013703,GSM5669081,right cerebral cortex,NaN
4348,SRR16808739,Bisulfite-Seq,SINGLE,SRP013703,GSM5667532,olfactory bulb,adult
4349,SRR16814931,Bisulfite-Seq,SINGLE,SRP013703,GSM5669986,cerebellum,adult
4350,SRR16808574,DNase-Hypersensitivity,PAIRED,SRP013703,GSM5667489,heart,NaN


In [81]:
## merge geo id, sra id together
## generate sample name

geo_sra_merge_dict = {}
sample_name_dict = {}
sample_name_dict['heart'] = {}
sample_name_dict['liver'] = {}
for tissue in ['heart', 'liver'] :
    geo_sra_merge_dict[tissue] = pd.merge(data_geo_dict[tissue], sra_meta, left_on = 'GSMID', right_on = 'GEO_Accession (exp)')
    for stage in geo_sra_merge_dict[tissue]['Stage'].unique() :
        df = geo_sra_merge_dict[tissue].loc[geo_sra_merge_dict[tissue]['Stage'] == stage].reset_index()
        for k,data in enumerate(df['Data'].unique()) :
            sub_df = df.loc[df['Data'] == data].reset_index()            
            for j,gsm in enumerate(sub_df['GSMID'].unique()) :
                sub_sub_df = sub_df.loc[sub_df['GSMID'] == gsm].drop('level_0', axis = 1).reset_index()
                data_type = data if data == 'WGBS' else data.split(' ')[0] if data[-8:] == "ChIP-seq" else "RNASeq"
                sample_name_dict[tissue][gsm] = f"{stage.split()[0]}.{data_type}.rep{j+1}"
        
    sample_name_df = pd.DataFrame.from_dict(sample_name_dict[tissue], orient='index').reset_index()
    sample_name_df.columns = ['GSMID', 'sample_name']
    geo_sra_merge_dict[tissue] = pd.merge(geo_sra_merge_dict[tissue], sample_name_df, on = 'GSMID')
    geo_sra_merge_dict[tissue].to_csv(f'geo_sra_id_{tissue}.csv', sep = ",", index = False, header = True)

geo_sra_merge_dict[tissue] 

,Stage,Data,GSMID,GSEID,Run,Assay Type,LibraryLayout,SRA Study,GEO_Accession (exp),Tissue,dev_stage,sample_name
0,E11.5 Liver,H3K9me3 ChIP-seq,GSM2192347,GSE82974,SRR3653117,ChIP-Seq,SINGLE,SRP013703,GSM2192347,liver,embryonic,E11.5.H3K9me3.rep1
1,E11.5 Liver,H3K9me3 ChIP-seq,GSM2192347,GSE82974,SRR3653118,ChIP-Seq,SINGLE,SRP013703,GSM2192347,liver,embryonic,E11.5.H3K9me3.rep1
2,E11.5 Liver,H3K9me3 ChIP-seq,GSM2192348,GSE82974,SRR3653119,ChIP-Seq,SINGLE,SRP013703,GSM2192348,liver,embryonic,E11.5.H3K9me3.rep2
3,E11.5 Liver,H3K9me3 ChIP-seq,GSM2192348,GSE82974,SRR3653120,ChIP-Seq,SINGLE,SRP013703,GSM2192348,liver,embryonic,E11.5.H3K9me3.rep2
4,E12.5 Liver,H3K9me3 ChIP-seq,GSM2191248,GSE82470,SRR3651489,ChIP-Seq,SINGLE,SRP013703,GSM2191248,liver,embryonic,E12.5.H3K9me3.rep1
...,...,...,...,...,...,...,...,...,...,...,...,...
69,E16.5 Liver,WGBS,GSM2191215,GSE82456,SRR3651455,Bisulfite-Seq,SINGLE,SRP013703,GSM2191215,liver,embryonic,E16.5.WGBS.rep4
70,P0 Liver,WGBS,GSM2191921,GSE82780,SRR3652380,Bisulfite-Seq,SINGLE,SRP013703,GSM2191921,liver,postnatal,P0.WGBS.rep1
71,P0 Liver,WGBS,GSM2191922,GSE82780,SRR3652381,Bisulfite-Seq,SINGLE,SRP013703,GSM2191922,liver,postnatal,P0.WGBS.rep2
72,P0 Liver,WGBS,GSM2191923,GSE82780,SRR3652382,Bisulfite-Seq,SINGLE,SRP013703,GSM2191923,liver,postnatal,P0.WGBS.rep3


In [17]:
## sra list for raw data downloading

for tissue in ['heart', 'liver'] :
    pd.DataFrame(geo_sra_merge_dict[tissue]['Run'].unique()).to_csv(f'download_sra_list_{tissue}.txt', header = False, index = False)

In [114]:
## generate soft link commands

for tissue in ['heart', 'liver'] :
    print("\n")
    cmd1 = f'cd ~/CHMsInOtherContexts/CellStateTransition/{tissue[0].upper()}{tissue[1:]}Development'
    print(cmd1)
    for gsm in geo_sra_merge_dict[tissue]['GSMID'].unique().tolist() :
        layout = geo_sra_merge_dict[tissue].loc[geo_sra_merge_dict[tissue]['GSMID'] == gsm]['LibraryLayout'].unique()[0]
        srr_ls = geo_sra_merge_dict[tissue].loc[geo_sra_merge_dict[tissue]['GSMID'] == gsm]['Run'].tolist()
        sampleN = geo_sra_merge_dict[tissue].loc[geo_sra_merge_dict[tissue]['GSMID'] == gsm]['sample_name'].tolist()[0]
        cmd2 = f'mkdir {sampleN} && cd {sampleN}'
        print(cmd2)
        if len(srr_ls) == 1 :
            if layout == 'SINGLE' :
                cmd3 = f"ln -sf ../0_raw_data/{srr_ls[0]}.fastq.gz {sampleN}.fq.gz && cd ../"
            else :
                cmd3 = f"ln -sf ../0_raw_data/{srr_ls[0]}_1.fastq.gz {sampleN}_R1.fq.gz && ln -sf ../0_raw_data/{srr_ls[0]}_2.fastq.gz {sampleN}_R2.fq.gz && cd ../"
            print(cmd3)
        else :
            for i,srr in enumerate(srr_ls) :
                if layout == 'SINGLE' :
                    cmd3 = f"ln -sf ../0_raw_data/{srr}.fastq.gz {sampleN}_part{i+1}.fq.gz"
                else :
                    cmd3 = f"ln -sf ../0_raw_data/{srr}_1.fastq.gz {sampleN}_part{i+1}_R1.fq.gz && ln -sf ../0_raw_data/{srr}_2.fastq.gz {sampleN}_part{i+1}_R2.fq.gz"
                cmd3 = f"{cmd3} && cd ../" if i == len(srr_ls)-1 else cmd3
                print(cmd3)
                
            if layout == 'SINGLE' :
                cmd4 = f'cat {sampleN}/{sampleN}_part*.fq.gz > {sampleN}/{sampleN}.fq.gz && rm {sampleN}/{sampleN}_part*.fq.gz'
            else :
                cmd4 = f'cat {sampleN}/{sampleN}_part*_R1.fq.gz > {sampleN}/{sampleN}_R1.fq.gz && cat {sampleN}/{sampleN}_part*_R2.fq.gz > {sampleN}/{sampleN}_R2.fq.gz && rm {sampleN}/{sampleN}_part*_R[12].fq.gz'
                      
            print(cmd4)



cd ~/CHMsInOtherContexts/CellStateTransition/HeartDevelopment
mkdir E10.5.WGBS.rep1 && cd E10.5.WGBS.rep1
ln -sf ../0_raw_data/SRR5833022.fastq.gz E10.5.WGBS.rep1.fq.gz && cd ../
mkdir E10.5.WGBS.rep2 && cd E10.5.WGBS.rep2
ln -sf ../0_raw_data/SRR5833023.fastq.gz E10.5.WGBS.rep2.fq.gz && cd ../
mkdir E10.5.WGBS.rep3 && cd E10.5.WGBS.rep3
ln -sf ../0_raw_data/SRR5833024.fastq.gz E10.5.WGBS.rep3.fq.gz && cd ../
mkdir E10.5.WGBS.rep4 && cd E10.5.WGBS.rep4
ln -sf ../0_raw_data/SRR5833025.fastq.gz E10.5.WGBS.rep4.fq.gz && cd ../
mkdir E11.5.WGBS.rep1 && cd E11.5.WGBS.rep1
ln -sf ../0_raw_data/SRR3652576.fastq.gz E11.5.WGBS.rep1.fq.gz && cd ../
mkdir E11.5.WGBS.rep2 && cd E11.5.WGBS.rep2
ln -sf ../0_raw_data/SRR3652577.fastq.gz E11.5.WGBS.rep2.fq.gz && cd ../
mkdir E11.5.WGBS.rep3 && cd E11.5.WGBS.rep3
ln -sf ../0_raw_data/SRR3652578.fastq.gz E11.5.WGBS.rep3.fq.gz && cd ../
mkdir E11.5.WGBS.rep4 && cd E11.5.WGBS.rep4
ln -sf ../0_raw_data/SRR3652579.fastq.gz E11.5.WGBS.rep4.fq.gz && cd ../


In [79]:
geo_sra_merge_dict['liver'].loc[geo_sra_merge_dict[tissue]['Run'] == 'SRR3651222'] 

,Stage,Data,GSMID,GSEID,Run,Assay Type,LibraryLayout,SRA Study,GEO_Accession (exp),Tissue,dev_stage,sample_name
44,E11.5 Liver,WGBS,GSM2190976,GSE82367,SRR3651222,Bisulfite-Seq,SINGLE,SRP013703,GSM2190976,liver,embryonic,E11.5.WGBS.rep1


In [115]:
for tissue in ['heart', 'liver'] :
    print("\n")
    for gsm in sample_name_dict[tissue].keys() :
        sample_name = sample_name_dict[tissue][gsm]
        datatype = sample_name.split('.')[-2]
        if datatype == 'WGBS' :
            cmd5 = f'WGBS_processing_publicData.sh ~/CHMsInOtherContexts/CellStateTransition/{tissue[0].upper()}{tissue[1:]}Development GEO {gsm} SINGLE {sample_name} mm10'
            print(cmd5)
        elif datatype == 'H3K9me3' or datatype == 'input' :
            cmd5 = f'ChIPseq_processing_publicData.sh ~/CHMsInOtherContexts/CellStateTransition/{tissue[0].upper()}{tissue[1:]}Development GEO {gsm} SINGLE {sample_name} mm10'
            print(cmd5)



WGBS_processing_publicData.sh ~/CHMsInOtherContexts/CellStateTransition/HeartDevelopment GEO GSM2701081 SINGLE E10.5.WGBS.rep1 mm10
WGBS_processing_publicData.sh ~/CHMsInOtherContexts/CellStateTransition/HeartDevelopment GEO GSM2701082 SINGLE E10.5.WGBS.rep2 mm10
WGBS_processing_publicData.sh ~/CHMsInOtherContexts/CellStateTransition/HeartDevelopment GEO GSM2701083 SINGLE E10.5.WGBS.rep3 mm10
WGBS_processing_publicData.sh ~/CHMsInOtherContexts/CellStateTransition/HeartDevelopment GEO GSM2701084 SINGLE E10.5.WGBS.rep4 mm10
ChIPseq_processing_publicData.sh ~/CHMsInOtherContexts/CellStateTransition/HeartDevelopment GEO GSM2308518 SINGLE E10.5.H3K9me3.rep1 mm10
ChIPseq_processing_publicData.sh ~/CHMsInOtherContexts/CellStateTransition/HeartDevelopment GEO GSM2308519 SINGLE E10.5.H3K9me3.rep2 mm10
ChIPseq_processing_publicData.sh ~/CHMsInOtherContexts/CellStateTransition/HeartDevelopment GEO GSM2308570 SINGLE E10.5.input.rep1 mm10
ChIPseq_processing_publicData.sh ~/CHMsInOtherContexts/Cel